# ADP1 Expression-Constrained Flux Analysis

## Objectives

This notebook performs expression-constrained flux balance analysis on 8 ADP1 strains using proteomics data. For each strain, we analyze flux profiles both with and without the DGOA enzyme pathway.

### Analysis Goals:
1. Generate predicted flux profiles for all 8 strains (16 conditions total: 8 strains × 2 DGOA variants)
2. Validate that all solutions are biologically feasible
3. Export results in multiple formats (JSON, Excel, Escher maps)
4. Compare metabolic differences between strains and DGOA conditions

### Strains:
- **ACN2586**: Initial construct with dgoA* and native DAHP synthesis deleted
- **ACN2821**: Evolved strain with single copy of dgoA*
- **ACN3015**: Single copy of dgoA after evolution
- **ACN3468**: Multiple copies of dgoA*
- **ACN3471**: Multiple copies of dgoA
- **ACN3474**: Multiple copies of dgoA, partially evolved
- **ACN3477**: Multiple copies of dgoA, more evolved
- **ADP1**: Wild-type Acinetobacter baylyi ADP1

## Run Expression Flux Analysis Pipeline

This step:
1. Loads the metabolic model and media
2. Loads proteomics data and averages replicates
3. Processes all 16 conditions (8 strains × 2 DGOA variants)
4. Caches intermediate results

In [ ]:
%run util.py
#Loading expression data and averaging replicates
raw_expression = MSExpression.from_spreadsheet(
    filename="data/ASCR_UGA_Proteomics_DgoA_add_strains_2025_pyruvate.xlsx",
    sheet_name="Imputed",
    skiprows=0,
    type="Log2",
    id_column="ACIAD"
)
#Averaging replicates
strains = ["Pyruvate_ACN2586_DgoA_2025", "Pyruvate_ACN2821_DgoA_2025", "Pyruvate_ACN3425_DgoA_2025", "Pyruvate_ACN3427_DgoA_2025","Pyruvate_ACN3429_DgoA_2025", "Pyruvate_ACN3430_DgoA_2025", "Pyruvate_ADP1_DgoA_2025"]
util.save("strains",strains)
averaged_expression = raw_expression.average_expression_replicates(strains)
util.save("averaged_expression_data", averaged_expression._data.to_dict())
#Transforming data
transformed_expression = averaged_expression.translate_data("RelativeAbundance")
util.save("transformed_expression_data", transformed_expression._data.to_dict())
#Translating to reaction data
model = MSModelUtil.from_cobrapy("data/FullyTranslatedPublishedModel.json")
reaction_expression = transformed_expression.build_reaction_expression(model.model)
util.save("reaction_expression_data", reaction_expression._data.to_dict())

/Users/chenry/Dropbox/Projects/KBUtilLib/src


2025-11-14 12:28:09,785 - __main__.NotebookUtil - INFO - Loaded 0 tokens from /Users/chenry/.tokens
2025-11-14 12:28:09,785 - __main__.NotebookUtil - INFO - Loaded kbase tokens from /Users/chenry/.kbase/token
2025-11-14 12:28:09,786 - __main__.NotebookUtil - WARNING - Section 'ModelSEEDBiochem' not found in config


modelseedpy 0.4.2
cobrakbase 0.4.0


2025-11-14 12:28:14,011 - __main__.NotebookUtil - INFO - ModelSEED database loaded from /Users/chenry/Dropbox/Projects/KBUtilLib/src/kbutillib/dependencies/ModelSEEDDatabase
2025-11-14 12:28:14,502 - __main__.NotebookUtil - WARNING - BLAST tools not found. Install NCBI BLAST+ to use BLAST functionality. On Ubuntu/Debian: sudo apt-get install ncbi-blast+, On MacOS: brew install blast
2025-11-14 12:28:14,503 - __main__.NotebookUtil - INFO - Notebook environment detected
2025-11-14 12:28:14,503 - __main__.NotebookUtil - INFO - ArgoGatewayClient initialised | model=gpto3mini env=dev timeout=120.0s url=https://apps-dev.inside.anl.gov/argoapi/api/v1/resource/streamchat/
/opt/anaconda3/envs/modelseed/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [2]:
results = {}
models_dict = {}
model.model.reactions.get_by_id("rxn01332_c0").lower_bound = 0
model.model.reactions.get_by_id("rxn01332_c0").upper_bound = 0
model.model.reactions.get_by_id("rxn00595_c0").lower_bound = 0
model.model.reactions.get_by_id("rxn00595_c0").upper_bound = 0
model.model.reactions.get_by_id("rxn00966_c0").lower_bound = 0
model.model.reactions.get_by_id("rxn00966_c0").upper_bound = 0
model.model.reactions.get_by_id("rxn12374_c0").lower_bound = 0
model.model.reactions.get_by_id("rxn12374_c0").upper_bound = 0
model.model.reactions.get_by_id("rxn12199_c0").lower_bound = 0
model.model.reactions.get_by_id("rxn12199_c0").upper_bound = 0
model.model.reactions.get_by_id("rxn12363_c0").lower_bound = 0
model.model.reactions.get_by_id("rxn12363_c0").upper_bound = 0
pyruvate_media = util.get_media("KBaseMedia/Carbon-Pyruvic-Acid")
solution = util.run_fba(model,media=pyruvate_media,objective="GROWTH_DASH_RXN",run_pfba=True)

# Constrain biomass to specified fraction of optimal
util.constrain_objective_to_fraction_of_optimum(
    model,
    objective="GROWTH_DASH_RXN",
    fraction=0.25
)
for i, strain in enumerate(strains, 1):
    model.util = util
    # Fit model flux to expression data
    analysis_result = reaction_expression.fit_model_flux_to_data(
        model=model,
        condition=strain,
        default_coef=0.01,
        activation_threshold=None,
        deactivation_threshold=0.00001
    )

    # Extract off_off list (reactions to be deactivated)
    off_off_list = analysis_result.get("off_off", [])

    # Extract on_on list (reactions to be activated)
    off_on_list = analysis_result.get("off_on", [])

    # Apply deactivation constraints
    for rxn_id in off_off_list:
        try:
            rxn = model_copy.model.reactions.get_by_id(rxn_id)
            rxn.lower_bound = 0
            rxn.upper_bound = 0
        except:
            # Reaction might not exist in model
            pass

    new_solution = util.run_fba(model,media=pyruvate_media,objective="GROWTH_DASH_RXN",run_pfba=True)   
    
    # Extract fluxes (only non-zero)
    fluxes = {}
    for rxn in model.model.reactions:
        if abs(new_solution.fluxes[rxn.id]) > 1e-9:
            fluxes[rxn.id] = new_solution.fluxes[rxn.id]
    dgoa_flux = new_solution.fluxes["DgoA"]
    biomass = new_solution.fluxes.get("GROWTH_DASH_RXN", 0)

    results[strain] = {
        "fluxes": fluxes,
        "biomass": biomass,
        "active_reactions": len(fluxes),
        "off_off_reactions": off_off_list,
        "off_on_reactions": off_on_list,
        "dgoa_flux": dgoa_flux,
        "solution_status": new_solution.status
    }

util.save("ADP1ExpressionAnalysis",results)

INFO:modelseedpy.core.msmodelutl:cpd00063 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd10516 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00205 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00254 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00099 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00058 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00030 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00034 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00063 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd10516 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00205 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00254 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00099 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00058 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00030 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00034 not found in model!


On: {}
Off: {'rxn03887_c0': 0.8933674222059408, 'rxn00990_c0': 0.9147331776101639, 'rxn01299_c0': 0.6973454881289313, 'rxn00327_c0': 0.5346019040157056, 'rxn03908_c0': 0.7178840646917763, 'rxn08783_c0': 0.933221657304459, 'rxn00470_c0': 0.9524091566771169, 'rxn12397_c0': 0.442841250950782, 'rxn01015_c0': 0.5172297676750564, 'rxn02296_c0': 0.38318482481009974, 'rxn02971_c0': 0.04184855110293412, 'rxn00101_c0': 0.13530360020315513, 'rxn05552_c0': 0.83334070045295, 'rxn12332_c0': 0.6812945329813767, 'rxn01368_c0': 0.233019394315495, 'rxn19072_c0': 0.6973454881289313, 'rxn05172_c0': 0.600449681232318, 'rxn00500_c0': 0.933221657304459, 'rxn12262_c0': 0.9173916865889098, 'rxn00966_c0': 0.8815212574144073, 'rxn03130_c0': 0.68204211234313, 'rxn12363_c0': 0.2814900156963053, 'rxn01545_c0': 0.6973454881289313, 'CYT_DASH_UBIQUINOL_DASH_OXID_DASH_D_DASH_RXN': 0.3017225414300364, 'rxn12450_c0': 0.6470425970101281, 'rxn12433_c0': 0.442841250950782, 'TRANS_DASH_RXNIM_DASH_5771': 0.600449681232318, 'r

INFO:modelseedpy.core.msmodelutl:cpd00063 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd10516 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00205 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00254 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00099 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00058 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00030 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00034 not found in model!


On: {}
Off: {'rxn03887_c0': 0.6421027836640113, 'rxn00990_c0': 0.8707579585042646, 'rxn01299_c0': 0.42867939071383643, 'rxn00327_c0': 0.8900950912702993, 'rxn03908_c0': 0.6533833700077213, 'rxn20547_c0': 0.42053420805264596, 'rxn00469_c0': 0.3786814954099409, 'rxn08783_c0': 0.9319755902712703, 'rxn12364_c0': 0.18652639705782076, 'rxn00470_c0': 0.9286554580246219, 'rxn01015_c0': 0.0791197310419166, 'rxn00101_c0': 0.33696941703632055, 'rxn05552_c0': 0.919174065840587, 'rxn12332_c0': 0.7112077913665861, 'rxn03159_c0': 0.13570768125272256, 'rxn19072_c0': 0.42867939071383643, 'rxn05172_c0': 0.7279890950261905, 'rxn00500_c0': 0.9319755902712703, 'rxn12262_c0': 0.7983631312955902, 'rxn00966_c0': 0.9705370317752477, 'rxn03130_c0': 0.775526496661107, 'rxn12363_c0': 0.1640366032811799, 'rxn01545_c0': 0.42867939071383643, 'rxn12450_c0': 0.7298561798967977, 'TRANS_DASH_RXNIM_DASH_5771': 0.7279890950261905, 'rxn00872_c0': 0.9090645147209278, 'rxn09167_c0': 0.7816670168976317, 'rxn12263_c0': 0.79836

INFO:modelseedpy.core.msmodelutl:cpd00063 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd10516 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00205 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00254 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00099 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00058 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00030 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00034 not found in model!


...

On: {}
Off: {'rxn03887_c0': 0.2518888205229927, 'rxn00990_c0': 0.9465223364296933, 'rxn01299_c0': 0.3836685171467996, 'rxn00327_c0': 0.7265922988293578, 'rxn03908_c0': 0.7891508363775667, 'rxn20547_c0': 0.3623436416608531, 'rxn08783_c0': 0.9610394483729865, 'rxn12364_c0': 0.40774112135408513, 'rxn00470_c0': 0.9517379993040634, 'rxn01015_c0': 0.31478075256556914, 'rxn02296_c0': 0.22009866033863845, 'rxn00101_c0': 0.20860594560856793, 'rxn05552_c0': 0.9064468232906078, 'rxn12332_c0': 0.8202149944703179, 'rxn03159_c0': 0.2320392493857883, 'rxn19072_c0': 0.3836685171467996, 'rxn05172_c0': 0.8749847929593239, 'rxn00500_c0': 0.9610394483729865, 'rxn12262_c0': 0.8459988570631671, 'rxn00966_c0': 0.9634728175382927, 'rxn03130_c0': 0.5368048162519918, 'rxn12223_c0': 0.03254235917584096, 'rxn12363_c0': 0.2900821445653568, 'rxn01545_c0': 0.3836685171467996, 'rxn12450_c0': 0.8098593056055454, 'TRANS_DASH_RXNIM_DASH_5771': 0.8749847929593239, 'rxn02927_c0': 0.656122835500985, 'rxn00872_c0': 0.937182

INFO:modelseedpy.core.msmodelutl:cpd00063 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd10516 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00205 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00254 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00099 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00058 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00030 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00034 not found in model!


On: {}
Off: {'rxn03887_c0': 0.5839599716202366, 'rxn00990_c0': 0.7800953687676427, 'rxn01299_c0': 0.2565626498125197, 'rxn00327_c0': 0.7502075616257093, 'rxn03908_c0': 0.87536067701107, 'rxn20547_c0': 0.3055013548346933, 'rxn08783_c0': 0.9676426634837674, 'rxn12364_c0': 0.2871450833486335, 'rxn00470_c0': 0.9214339609839828, 'rxn01015_c0': 0.24416358235946944, 'rxn02296_c0': 0.09542890333745568, 'rxn00101_c0': 0.4243932219517357, 'rxn05552_c0': 0.9363816987944993, 'rxn12332_c0': 0.7532671745496913, 'rxn03159_c0': 0.44171916661394584, 'rxn19072_c0': 0.2565626498125197, 'rxn05172_c0': 0.9230480262537752, 'rxn00500_c0': 0.9676426634837674, 'rxn12262_c0': 0.6735602763226768, 'rxn00966_c0': 0.9789875979555612, 'rxn03130_c0': 0.6904957238848985, 'rxn12223_c0': 0.28718230274046275, 'rxn12363_c0': 0.25314472139146743, 'rxn01545_c0': 0.2565626498125197, 'rxn12450_c0': 0.7765622557669312, 'rxn01867_c0': 0.12747199954525473, 'TRANS_DASH_RXNIM_DASH_5771': 0.9230480262537752, 'rxn02927_c0': 0.695380

INFO:modelseedpy.core.msmodelutl:cpd00063 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd10516 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00205 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00254 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00099 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00058 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00030 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00034 not found in model!


On: {}
Off: {'rxn00990_c0': 0.8518682334921533, 'rxn01299_c0': 0.15974344666495485, 'rxn00327_c0': 0.7738243106099235, 'rxn03908_c0': 0.7330483247385104, 'rxn20547_c0': 0.18566345912020712, 'rxn08783_c0': 0.9300960054676595, 'rxn12364_c0': 0.16744011751078647, 'rxn00470_c0': 0.9514578278025722, 'rxn01015_c0': 0.0006165929287904392, 'rxn02296_c0': 0.016734150504007245, 'rxn02971_c0': 0.2589926049271115, 'rxn00101_c0': 0.15339558928214386, 'rxn05552_c0': 0.9106322538912686, 'rxn12332_c0': 0.7289934355114208, 'rxn03159_c0': 0.265648485894674, 'rxn19072_c0': 0.15974344666495485, 'rxn05172_c0': 0.8081906315467818, 'rxn00500_c0': 0.9300960054676595, 'rxn12262_c0': 0.5029278131562755, 'rxn00966_c0': 0.9069180467959839, 'rxn03130_c0': 0.6187816730165618, 'rxn01545_c0': 0.15974344666495485, 'rxn12450_c0': 0.7058757349270752, 'TRANS_DASH_RXNIM_DASH_5771': 0.8081906315467818, 'rxn02927_c0': 0.5689355940020657, 'rxn00872_c0': 0.9525109557321149, 'rxn09167_c0': 0.4190278780896897, 'rxn12263_c0': 0.

# Fitting fluxes to fold change data

In [ ]:
%run util.py
from modelseedpy.fbapkg.fluxfittingpkg import FluxFittingPkg

# Load the uplist and downlist
with open("data/uplist.txt", "r") as f:
    uplist = [line.strip() for line in f if line.strip()]

with open("data/downlist.txt", "r") as f:
    downlist = [line.strip() for line in f if line.strip()]

print(f"Loaded {len(uplist)} genes in uplist")
print(f"Loaded {len(downlist)} genes in downlist")

# Load the existing results
results = util.load("ADP1ExpressionAnalysis")
strains = util.load("strains")

# Load the model
model = MSModelUtil.from_cobrapy("data/FullyTranslatedPublishedModel.json")

# Build a mapping from genes to reactions
gene_to_reactions = {}
for rxn in model.model.reactions:
    for gene in rxn.genes:
        gene_id = str(gene.id)
        if gene_id not in gene_to_reactions:
            gene_to_reactions[gene_id] = []
        gene_to_reactions[gene_id].append(rxn.id)

print(f"Built gene-to-reaction mapping for {len(gene_to_reactions)} genes")

# For each strain, create targetflux and run flux fitting
for strain in strains:
    print(f"\nProcessing {strain}...")
    
    if strain not in results:
        print(f"  WARNING: {strain} not found in results, skipping")
        continue
    
    # Get original fluxes
    original_fluxes = results[strain]["fluxes"]
    
    # Create targetflux by modifying fluxes based on gene lists
    targetflux = {}
    modified_count = {"up": 0, "down": 0, "unchanged": 0}
    
    for rxn_id, flux_value in original_fluxes.items():
        # Check if this reaction is associated with uplist or downlist genes
        rxn_genes = []
        if rxn_id in model.model.reactions:
            rxn = model.model.reactions.get_by_id(rxn_id)
            rxn_genes = [str(g.id) for g in rxn.genes]
        
        # Determine if reaction should be modified
        is_upregulated = any(gene in uplist for gene in rxn_genes)
        is_downregulated = any(gene in downlist for gene in rxn_genes)
        
        if is_upregulated:
            targetflux[rxn_id] = flux_value * 2.0
            modified_count["up"] += 1
        elif is_downregulated:
            targetflux[rxn_id] = flux_value / 2.0
            modified_count["down"] += 1
        else:
            targetflux[rxn_id] = flux_value
            modified_count["unchanged"] += 1
    
    # Store targetflux in results
    results[strain]["targetflux"] = targetflux
    
    print(f"  Created targetflux: {modified_count['up']} up, {modified_count['down']} down, {modified_count['unchanged']} unchanged")
    
    # Create a fresh model copy for flux fitting
    import cobra.io
    model_copy = MSModelUtil.from_cobrapy(cobra.io.json.to_json(model.model))
    
    # Apply same constraints as original simulation
    model_copy.model.reactions.get_by_id("rxn01332_c0").lower_bound = 0
    model_copy.model.reactions.get_by_id("rxn01332_c0").upper_bound = 0
    model_copy.model.reactions.get_by_id("rxn00595_c0").lower_bound = 0
    model_copy.model.reactions.get_by_id("rxn00595_c0").upper_bound = 0
    model_copy.model.reactions.get_by_id("rxn00966_c0").lower_bound = 0
    model_copy.model.reactions.get_by_id("rxn00966_c0").upper_bound = 0
    model_copy.model.reactions.get_by_id("rxn12374_c0").lower_bound = 0
    model_copy.model.reactions.get_by_id("rxn12374_c0").upper_bound = 0
    model_copy.model.reactions.get_by_id("rxn12199_c0").lower_bound = 0
    model_copy.model.reactions.get_by_id("rxn12199_c0").upper_bound = 0
    model_copy.model.reactions.get_by_id("rxn12363_c0").lower_bound = 0
    model_copy.model.reactions.get_by_id("rxn12363_c0").upper_bound = 0
    
    # Relax growth constraints (remove biomass fraction constraint)
    biomass_rxn = model_copy.model.reactions.get_by_id("GROWTH_DASH_RXN")
    biomass_rxn.lower_bound = 0  # Allow any positive growth
    
    # Set up media
    pyruvate_media = util.get_media("KBaseMedia/Carbon-Pyruvic-Acid")
    util._set_model_media(model_copy, pyruvate_media)
    
    # Use FluxFittingPkg to fit to targetflux
    try:
        # Get package manager
        if not hasattr(model_copy, 'pkgmgr'):
            from modelseedpy import MSPackageManager
            model_copy.pkgmgr = MSPackageManager.get_pkg_mgr(model_copy.model)
        
        # Build flux fitting package
        ffpkg = model_copy.pkgmgr.getpkg("FluxFittingPkg")
        ffpkg.build_package({
            "target_flux": targetflux,
            "set_objective": 1
        })
        
        # Optimize
        solution = model_copy.model.optimize()
        
        if solution.status == "optimal":
            # Extract new fluxes
            newfluxes = {}
            for rxn in model_copy.model.reactions:
                if abs(solution.fluxes[rxn.id]) > 1e-9:
                    newfluxes[rxn.id] = solution.fluxes[rxn.id]
            
            results[strain]["newfluxes"] = newfluxes
            results[strain]["newflux_biomass"] = solution.fluxes.get("GROWTH_DASH_RXN", 0)
            results[strain]["newflux_status"] = solution.status
            
            print(f"  ✓ Flux fitting successful: {len(newfluxes)} active reactions, biomass={results[strain]['newflux_biomass']:.4f}")
        else:
            print(f"  ✗ Flux fitting failed: {solution.status}")
            results[strain]["newfluxes"] = {}
            results[strain]["newflux_biomass"] = 0
            results[strain]["newflux_status"] = solution.status
    
    except Exception as e:
        print(f"  ✗ Error during flux fitting: {str(e)}")
        import traceback
        traceback.print_exc()
        results[strain]["newfluxes"] = {}
        results[strain]["newflux_biomass"] = 0
        results[strain]["newflux_status"] = "error"

# Save updated results
util.save("ADP1ExpressionAnalysis", results)
print(f"\n✓ Saved updated results with targetflux and newfluxes for all strains")

## Generate Outputs

Save results in multiple formats:
1. Individual JSON files for each condition
2. Summary JSON with metadata
3. Multi-sheet Excel workbook
4. Escher metabolic maps (HTML)

In [ ]:
# Save individual flux JSON files
print("Saving individual flux files...")
for condition_key, result_data in results.items():
    fluxes = result_data.get("fluxes", {})
    util.save(f"{condition_key}_fluxes", fluxes)
print(f"Saved {len(results)} flux JSON files to datacache/")

In [1]:
%run util.py
model = MSModelUtil.from_cobrapy("data/FullyTranslatedPublishedModel.json")
results = util.load("ADP1ExpressionAnalysis")
# Generate Escher maps with "full" map
print("Generating Escher metabolic maps...")
output = util.create_map_html(
    model=model.model if hasattr(model, 'model') else model,
    flux_solution=results["ACN2586"]["fluxes"],
    map_name="full",
    map_json="data/full.json",
    output_file="Test.html",
    title="Test"
)


/Users/chenry/Dropbox/Projects/KBUtilLib/src


2025-11-11 14:52:14,037 - __main__.NotebookUtil - INFO - Loaded 0 tokens from /Users/chenry/.tokens
2025-11-11 14:52:14,038 - __main__.NotebookUtil - INFO - Loaded kbase tokens from /Users/chenry/.kbase/token
2025-11-11 14:52:14,038 - __main__.NotebookUtil - WARNING - Section 'ModelSEEDBiochem' not found in config


modelseedpy 0.4.2
cobrakbase 0.4.0


2025-11-11 14:52:18,090 - __main__.NotebookUtil - INFO - ModelSEED database loaded from /Users/chenry/Dropbox/Projects/KBUtilLib/src/kbutillib/dependencies/ModelSEEDDatabase
2025-11-11 14:52:18,567 - __main__.NotebookUtil - WARNING - BLAST tools not found. Install NCBI BLAST+ to use BLAST functionality. On Ubuntu/Debian: sudo apt-get install ncbi-blast+, On MacOS: brew install blast
2025-11-11 14:52:18,568 - __main__.NotebookUtil - INFO - Notebook environment detected
2025-11-11 14:52:18,568 - __main__.NotebookUtil - INFO - ArgoGatewayClient initialised | model=gpto3mini env=dev timeout=120.0s url=https://apps-dev.inside.anl.gov/argoapi/api/v1/resource/streamchat/
2025-11-11 14:52:18,749 - __main__.NotebookUtil - INFO - Using enhanced arrow visualization with smart defaults
2025-11-11 14:52:18,749 - __main__.NotebookUtil - INFO - Generated smart reaction_scale with 5 stops
2025-11-11 14:52:18,864 - __main__.NotebookUtil - INFO - Successfully got HTML from builder.save_html()
2025-11-11

Generating Escher metabolic maps...


In [3]:
%run util.py
# Generate Escher map directly using escher.Builder (first strain only)
import escher
from escher import Builder
import json
import os

# Load model and results
model = MSModelUtil.from_cobrapy("data/FullyTranslatedPublishedModel.json")
results = util.load("ADP1ExpressionAnalysis")
strains = ["ACN2586", "ACN2821", "ACN3015", "ACN3468", "ACN3471", "ACN3474", "ACN3477", "ADP1"]

# Get first strain
strain = strains[0]
print(f"Generating map for {strain}...")

# Get fluxes for this strain
fluxes = results[strain].get("fluxes", {})
if not fluxes:
    print(f"Error: No flux data for {strain}")
else:
    print(f"Found {len(fluxes)} reactions with flux data")
    
    # Load the map JSON file
    map_json_path = "data/full.json"
    if not os.path.exists(map_json_path):
        print(f"Warning: {map_json_path} not found. Trying to use map_name instead.")
        map_json_string = None
        map_name = "full"
    else:
        # Load as dict first, then convert to JSON string (Builder expects string, not dict)
        with open(map_json_path, 'r') as f:
            map_json_dict = json.load(f)
        map_json_string = json.dumps(map_json_dict)  # Convert to JSON string
        map_name = None
        print(f"Loaded map from {map_json_path}")
    
    # Create Builder
    builder = Builder(
        model=model.model,
        map_json=map_json_string if map_json_string else None,
        map_name=map_name if not map_json_string else None,
        reaction_data=fluxes,
        height=800,
        width=1200
    )
    
    # Save HTML file
    output_file = f"nboutput/escher_{strain}_direct.html"
    os.makedirs("nboutput", exist_ok=True)
    builder.save_html(output_file)
    print(f"✓ Created: {output_file}")


2025-11-07 00:10:26,934 - __main__.NotebookUtil - INFO - Loaded 0 tokens from /Users/chenry/.tokens
2025-11-07 00:10:26,935 - __main__.NotebookUtil - INFO - Loaded kbase tokens from /Users/chenry/.kbase/token
2025-11-07 00:10:26,936 - __main__.NotebookUtil - WARNING - Section 'ModelSEEDBiochem' not found in config
2025-11-07 00:10:26,936 - __main__.NotebookUtil - INFO - ModelSEED database loaded from /Users/chenry/Dropbox/Projects/KBUtilLib/src/kbutillib/dependencies/ModelSEEDDatabase


/Users/chenry/Dropbox/Projects/KBUtilLib/src


2025-11-07 00:10:27,296 - __main__.NotebookUtil - INFO - Notebook environment detected
2025-11-07 00:10:27,297 - __main__.NotebookUtil - INFO - ArgoGatewayClient initialised | model=gpto3mini env=dev timeout=120.0s url=https://apps-dev.inside.anl.gov/argoapi/api/v1/resource/streamchat/


Generating map for ACN2586...
Found 433 reactions with flux data
Loaded map from data/full.json
✓ Created: nboutput/escher_ACN2586_direct.html
